## Setup

### Imports

In [4]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from src.api_eooh import *
from src.utils import *

%load_ext autoreload
%autoreload 2

/opt/conda/envs/aiahs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using device: {device}')

using device: cuda


#### Add 'split' col to dataset

In [12]:
%%script false --no-raise-error
# Read the dataset
dataset = pd.read_csv('data/hs_davidson2017.csv')

# Set the random seed for reproducibility
np.random.seed(42)

# Shuffle the dataset
shuffled_dataset = dataset.sample(frac=1, random_state=42)

# Calculate the exact number of rows for each split
num_train = int(0.8 * len(dataset))
num_val = int(0.1 * len(dataset))
# The remaining rows will be assigned to 'test'

# Assign 'train', 'val', or 'test' based on the row index
shuffled_dataset['split'] = 'test'
shuffled_dataset.iloc[:num_train, shuffled_dataset.columns.get_loc('split')] = 'train'
shuffled_dataset.iloc[num_train:num_train+num_val, shuffled_dataset.columns.get_loc('split')] = 'val'

# If you want the dataset in its original order, you can sort it by its index
dataset_with_split = shuffled_dataset.sort_index()

# Check the distribution of the splits
print(dataset_with_split['split'].value_counts(normalize=False))

# Save the updated dataset if needed
dataset_with_split.to_csv('data/hs_davidson2017.csv', index=False)


split
train    19826
test      2479
val       2478
Name: count, dtype: int64


## Manage channels

In [8]:
# get paths
paths = get_paths()
# list channels 
my_channels = request_api(requests.get, paths.my_channels).json()
# print(my_channels)
channel = my_channels[0]
channel_path = f"/channel/{channel['uid']}"
channel_data = request_api(requests.get, channel_path).json()

channel name: LGBTQ, uid: 648dcd4b3708dcf1b65d70e9


### Download channel data